In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/cis/home/dpacker/my_documents/vote-counts


In [51]:
import pandas as pd
from pathlib import Path
import os
from src import preprocessing, mvp
from jax import random
from tqdm.notebook import tqdm
import json


os.environ['CUDA_VISIBLE_DEVICES'] = '3'

rngs = random.split(random.PRNGKey(1), 10)

path_data = Path("data")
path_raw_data = path_data / "raw_data"

In [61]:
raw_data = pd.read_csv(path_raw_data / "Local_2018.csv")
data = preprocessing.preprocess(raw_data)
elections_dict = preprocessing.get_elections_dict(data)
county_ids = elections_dict.keys()

SyntaxError: invalid decimal literal (1159727162.py, line 4)

In [62]:
len(elections_dict[list(county_ids)[18]])

3

In [63]:
county_pivot_odds = {}
for county_id in tqdm(county_ids):
  county_elections = elections_dict[county_id]
  if len(county_elections) > 0:
    pivot_odds = mvp.get_pivot_odds_from_elections_c2(rngs[0], county_elections)
    county_pivot_odds[int(county_id)] = pivot_odds


  0%|          | 0/1444 [00:00<?, ?it/s]

In [65]:
fp = open(path_data / "county_pivot_odds_c2_full.json", "w")
json.dump(county_pivot_odds, fp)

In [74]:
pd.DataFrame.from_dict(county_pivot_odds, orient="index", columns = ["pivot_odds"]).to_json(path_data / "county_pivot_odds_c2.csv")